# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

# Part 1

In [2]:
!pip install folium
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from folium import plugins
from folium.plugins import MarkerCluster

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 5.1 MB/s  eta 0:00:01
Libraries imported.


In [3]:
#scrape wiki website
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_text = requests.get(url).text
soup = BeautifulSoup(website_text,'xml')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
df.shape

(103, 3)

In [13]:
df.info

<bound method DataFrame.info of     PostalCode                 Borough  \
0          M3A              North York   
1          M4A              North York   
2          M5A        Downtown Toronto   
3          M6A              North York   
4          M7A            Queen's Park   
5          M9A               Etobicoke   
6          M1B             Scarborough   
7          M3B              North York   
8          M4B               East York   
9          M5B        Downtown Toronto   
10         M6B              North York   
11         M9B               Etobicoke   
12         M1C             Scarborough   
13         M3C              North York   
14         M4C               East York   
15         M5C        Downtown Toronto   
16         M6C                    York   
17         M9C               Etobicoke   
18         M1E             Scarborough   
19         M4E            East Toronto   
20         M5E        Downtown Toronto   
21         M6E                    York   
22

# End of Part 1

# Part 2

In [5]:
!pip install geocoder # This came as a recommendation. The original was !conda install -c conda-forge geocoder --yes

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 7.8 MB/s  eta 0:00:01


In [6]:
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [7]:
#get latitude and longitude using geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.arcgis('{}, Toronto, Ontario'.format('Postal Code'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print(latitude,longitude )

43.648690000000045 -79.38543999999996


In [8]:
#read CSV
url_csv='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
geotable = pd.read_csv(url_csv)
geotable.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#append lat & long into table[0]
toronto = pd.merge(df, geotable, left_on='PostalCode', right_on='Postal Code', left_index=False, right_index=False)
toronto.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


In [11]:
#remove the duplicated column
del toronto["Postal Code"]

In [12]:
#export my dataframe to CSV for me to upload to github later
toronto.to_csv(r'toronto_geo_data.csv')

In [13]:
toronto.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# End of Part 2

# Part 3

In [14]:
#The notebook from here includes the Clustering and the plotting of the neighbourhoods of Canada which contain Toronto in their Borough
#Getting all the rows from the data frame which contains Toronto in their Borough.
toronto_only = toronto[toronto['Borough'].str.contains('Toronto',regex=False)]
toronto_only.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [15]:
#Visualizing all the Neighbourhoods of the above toronto_only data frame using Folium
map_toronto = folium.Map(location=[43.64869, -79.38543],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(toronto_only['Latitude'],toronto_only['Longitude'],toronto_only['Borough'],toronto_only['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [16]:
# The code was removed by Watson Studio for sharing.

In [17]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [18]:
#Get neighborhood around Borough of Downtown Toronto
DownTor_latitude = toronto_only.loc[toronto_only['Borough']=='Downtown Toronto','Latitude'] # neighborhood latitude value
DownTor_longitude = toronto_only.loc[toronto_only['Borough']=='Downtown Toronto','Longitude'] # neighborhood longitude value

DownTor_neigh_name = toronto_only.loc[toronto_only['Borough']=='Downtown Toronto','Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(DownTor_neigh_name, 
                                                               DownTor_latitude, 
                                                               DownTor_longitude))


Latitude and longitude values of 2                             Regent Park, Harbourfront
9                              Garden District, Ryerson
15                                       St. James Town
20                                          Berczy Park
24                                   Central Bay Street
25                                             Christie
30                             Richmond, Adelaide, King
36    Harbourfront East, Union Station, Toronto Islands
42             Toronto Dominion Centre, Design Exchange
48                       Commerce Court, Victoria Hotel
80                       University of Toronto, Harbord
84            Kensington Market, Chinatown, Grange Park
87    CN Tower, King and Spadina, Railway Lands, Har...
91                                             Rosedale
96                          St. James Town, Cabbagetown
97               First Canadian Place, Underground city
99                                 Church and Wellesley
Name: Neighborh

In [19]:
#Get neighborhood Kensington Market, Chinatown, Grange Park
CN_latitude = toronto_only.loc[toronto_only['Neighborhood']=='Kensington Market, Chinatown, Grange Park','Latitude'] # neighborhood latitude value
CN_longitude = toronto_only.loc[toronto_only['Neighborhood']=='Kensington Market, Chinatown, Grange Park','Longitude'] # neighborhood longitude value

CN_neigh_name = toronto_only.loc[toronto_only['Neighborhood']=='Kensington Market, Chinatown, Grange Park','Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(CN_neigh_name, 
                                                               CN_latitude, 
                                                               CN_longitude))

Latitude and longitude values of 84    Kensington Market, Chinatown, Grange Park
Name: Neighborhood, dtype: object are 84    43.653206
Name: Latitude, dtype: float64, 84   -79.400049
Name: Longitude, dtype: float64.


In [20]:
#Get the top 100 venues that are in 'Kensington Market, Chinatown, Grange Park' within a radius of 500 meters.
#create the GET request URL. Name your URL url.

search_query = 'Kensington Market, Chinatown, Grange Park'
radius = 500
LIMIT=100
latitude = 43.653206
longitude = -79.400049
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [21]:
#get json file from FourSquare
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60d089d82f362615cb149397'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Kensington',
  'headerFullLocation': 'Kensington, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 58,
  'suggestedBounds': {'ne': {'lat': 43.6577060045, 'lng': -79.39384110909378},
   'sw': {'lat': 43.648705995499995, 'lng': -79.4062568909062}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50427a03e4b08d9f5931f593',
       'name': 'Seven Lives - Tacos y Mariscos',
       'location': {'address': '69 Kensington Ave',
        'crossStreet': 'Baldwin St',
        'lat': 43.65441817160925,
        'lng': -79.40054467846261,
        'labeledLatLng

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
#Get the venus into Pandas Dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Seven Lives - Tacos y Mariscos,Mexican Restaurant,43.654418,-79.400545
1,Banh Mi Nguyen Huong,Vietnamese Restaurant,43.653628,-79.398376
2,The Moonbean Cafe,Café,43.654147,-79.400182
3,Blackbird Baking Co,Bakery,43.654897,-79.400619
4,Kid Icarus,Arts & Crafts Store,43.653933,-79.401719


In [25]:
#Get the 500 venues that are in 'Kensington Market, Chinatown, Grange Park' within a radius of 500 meters.
#create the GET request URL. Name your URL url.

search_query = 'Kensington Market, Chinatown, Grange Park'
radius = 500
LIMIT=100
latitude = 43.653206
longitude = -79.400049


In [26]:
#Function to get nearby venue 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
CN_venues = getNearbyVenues(names=toronto_only['Neighborhood'],
                                   latitudes=toronto_only['Latitude'],
                                   longitudes=toronto_only['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [29]:
print(CN_venues.shape)
CN_venues.head()

(1495, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [30]:
CN_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,46,46,46,46,46,46
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,60,60,60,60,60,60
Christie,14,14,14,14,14,14
Church and Wellesley,68,68,68,68,68,68
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,27,27,27,27,27,27
Davisville North,10,10,10,10,10,10


In [31]:
#To find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(CN_venues['Venue Category'].unique())))

There are 220 uniques categories.


In [32]:
#Anaylze each neighborhood
# one hot encoding
CN_onehot = pd.get_dummies(CN_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
CN_onehot['Neighborhood'] = CN_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [CN_onehot.columns[-1]] + list(CN_onehot.columns[:-1])
CN_onehot = CN_onehot[fixed_columns]

CN_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [33]:
CN_onehot.shape

(1495, 220)

In [34]:
#Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
CN_grouped = CN_onehot.groupby('Neighborhood').mean().reset_index()
CN_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,0.021739,0.000000,0.000000,0.000000,0.0000,0.000000,0.021739,0.065217,0.000000,0.000000,0.00,0.021739,0.021739,0.000000,0.043478,0.000000,0.000000,0.021739,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.021739,0.000000,0.021739,0.000000,0.0000,0.000000,0.021739,0.000000,0.00000,0.000000,0.000000,0.000000,0.086957,0.065217,0.000000,0.000000,0.00,0.00,0.021739,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0

In [35]:
#confirmt the new group size
CN_grouped.shape

(39, 220)

In [36]:
#explore top 5 venue
num_top_venues = 5

for hood in CN_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =CN_grouped[CN_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Berczy Park----
                venue  freq
0        Cocktail Bar  0.09
1              Bakery  0.07
2      Sandwich Place  0.07
3         Coffee Shop  0.07
4  Seafood Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.09
1     Coffee Shop  0.09
2          Bakery  0.09
3  Breakfast Spot  0.09
4  Sandwich Place  0.09


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.19
1       Airport Lounge  0.12
2        Boat or Ferry  0.06
3      Harbor / Marina  0.06
4  Rental Car Location  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1       Sandwich Place  0.08
2     Sushi Restaurant  0.07
3  Japanese Restaurant  0.05
4                 Café  0.05


----Christie----
           venue  freq
0  Grocery Store  0.29
1           Café  0.21
2           Park  0.14
3    Cof

In [37]:
#Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
# new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = CN_grouped['Neighborhood']

for ind in np.arange(CN_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CN_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Sandwich Place,Coffee Shop,Bakery,Seafood Restaurant,Beer Bar,Farmers Market,Vegetarian / Vegan Restaurant,Japanese Restaurant,Jazz Club
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Sandwich Place,Breakfast Spot,Bakery,Intersection,Nightclub,Convenience Store,Restaurant,Japanese Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Boutique,Rental Car Location,Plane,Bar,Airport Terminal,Harbor / Marina,Airport Gate,Airport Food Court
3,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Japanese Restaurant,Italian Restaurant,Pizza Place,Salad Place,Burger Joint,Bank
4,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Distribution Center,Dance Studio


In [39]:
#k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

CN_grouped_clustering = CN_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CN_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [40]:
#Recheck toronto_only shape
toronto_only.shape

(39, 5)

In [41]:
#Recheck neighborhoods_venues_sorted shape
neighborhoods_venues_sorted.shape

(39, 11)

In [42]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

CN_merged = toronto_only

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
CN_merged = CN_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

CN_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Pub,Café,Restaurant,Dessert Shop,Electronics Store,Sandwich Place,Asian Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Sandwich Place,Clothing Store,Café,Thai Restaurant,Bank,Cosmetics Shop,Pizza Place,Japanese Restaurant,Hotel
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Cocktail Bar,Italian Restaurant,Clothing Store,Restaurant,Japanese Restaurant,Cosmetics Shop,Beer Bar,Department Store
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Music Venue,Wine Shop,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Cocktail Bar,Sandwich Place,Coffee Shop,Bakery,Seafood Restaurant,Beer Bar,Farmers Market,Vegetarian / Vegan Restaurant,Japanese Restaurant,Jazz Club


In [43]:
CN_merged['Cluster Labels'].unique()


array([2, 0, 3, 4, 1], dtype=int32)

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CN_merged['Latitude'], CN_merged['Longitude'], CN_merged['Neighborhood'], CN_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [45]:
#Examine Cluster 0
CN_merged.loc[CN_merged['Cluster Labels'] == 0, CN_merged.columns[[1] + list(range(5, CN_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,0,Health Food Store,Pub,Music Venue,Wine Shop,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
35,East York/East Toronto,0,Park,Convenience Store,Bus Stop,Intersection,Wine Shop,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [46]:
#Examine Cluster 1
CN_merged.loc[CN_merged['Cluster Labels'] == 1, CN_merged.columns[[1] + list(range(5, CN_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,1,Trail,Tennis Court,Summer Camp,Wine Shop,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [47]:
#Examine Cluster 2
CN_merged.loc[CN_merged['Cluster Labels'] == 2, CN_merged.columns[[1] + list(range(5, CN_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Coffee Shop,Park,Bakery,Pub,Café,Restaurant,Dessert Shop,Electronics Store,Sandwich Place,Asian Restaurant
9,Downtown Toronto,2,Coffee Shop,Sandwich Place,Clothing Store,Café,Thai Restaurant,Bank,Cosmetics Shop,Pizza Place,Japanese Restaurant,Hotel
15,Downtown Toronto,2,Coffee Shop,Café,Cocktail Bar,Italian Restaurant,Clothing Store,Restaurant,Japanese Restaurant,Cosmetics Shop,Beer Bar,Department Store
20,Downtown Toronto,2,Cocktail Bar,Sandwich Place,Coffee Shop,Bakery,Seafood Restaurant,Beer Bar,Farmers Market,Vegetarian / Vegan Restaurant,Japanese Restaurant,Jazz Club
24,Downtown Toronto,2,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Japanese Restaurant,Italian Restaurant,Pizza Place,Salad Place,Burger Joint,Bank
25,Downtown Toronto,2,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Distribution Center,Dance Studio
30,Downtown Toronto,2,Coffee Shop,Café,Sandwich Place,Gym,Clothing Store,Restaurant,Sushi Restaurant,Pizza Place,Burrito Place,Bank
31,West Toronto,2,Bakery,Pharmacy,Music Venue,Brewery,Bar,Bank,Middle Eastern Restaurant,Supermarket,Art Gallery,Café
36,Downtown Toronto,2,Coffee Shop,Café,Hotel,Scenic Lookout,Pizza Place,Aquarium,Park,Sandwich Place,Deli / Bodega,Gym
37,West Toronto,2,Bar,Vietnamese Restaurant,New American Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Café,Asian Restaurant,Yoga Studio,Restaurant,Bistro


In [48]:
#Examine Cluster 3
CN_merged.loc[CN_merged['Cluster Labels'] == 3, CN_merged.columns[[1] + list(range(5, CN_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,3,Bus Line,Swim School,Park,Business Service,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
91,Downtown Toronto,3,Park,Playground,Trail,Wine Shop,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [49]:
#Examine Cluster 4
CN_merged.loc[CN_merged['Cluster Labels'] == 4, CN_merged.columns[[1] + list(range(5, CN_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,4,Garden,Wine Shop,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


# End of Part 3